In [1]:
from lxml import etree
from dotenv import load_dotenv 
from openai import OpenAI

In [2]:
# Read XSD from 1872.1-2024
with open('app/resources/schema.xsd', 'r') as file:
    schema = file.read()

In [3]:
# Read XSD from 1872.1-2024
with open('app/resources/reza_small.geojson', 'r') as file:
    geojson = file.read()

In [6]:
# global environment config
load_dotenv('/Users/marcos/.gpt/token.env')
MAX_TOKENS: int = 2000

In [78]:
client = OpenAI()

In [79]:
messages = [
    {
        "role": "system",
        "content": "You are a mission planner that generates XML mission plans based on robotic task representation. \
            When asked to generate a mission, create an XML file conformant to the known schema and use the GeoJSON file to provide references in the mission plan for things such as GPS location, tree type, etc. \
            It must be syntactically correct and validate using an XML linter.",
    },
    # context
    {
        "role": "user",
        "content": "This is the schema for which you must generate mission plan XML documents:" + schema
    },
    {
        "role": "assistant",
        "content": "If you have any specific questions or modifications you'd like to discuss regarding this schema, feel free to ask!"
    },
    {
        "role": "user",
        "content": "This is the GeoJSON for which you must generate mission plan XML documents. This is our orchard:" + geojson
    },
    {
        "role": "assistant",
        "content": "Thank you for providing the GeoJSON file. I'll assist you in creating the XML file for your robotic mission plan when you provide your mission."
    },
    # TODO: add context of farm layout so that machine can generate XML with relevant state information
]

In [80]:
m = "Make a plan to take a picture of every other pistachio tree."
message = {"role": "user", "content": m}

In [81]:
messages.append(message)

In [82]:
def ask_gpt(messages):
    return client.chat.completions.create(
        model="gpt-4o", messages=messages, max_tokens=MAX_TOKENS
    )

In [83]:
completion = ask_gpt(messages)

In [84]:
with open('output.txt', 'w') as fp:
    fp.write(completion.choices[0].message.content)

In [85]:
gpt_response = completion.choices[0].message.content

xml_response = gpt_response.split("```xml\n")[1]
xml_response = xml_response.split("```")[0]

In [4]:
with open('app/gpt_outputs/gpt_example.xml', 'w') as fp:
    fp.write(xml_response)

NameError: name 'xml_response' is not defined

In [3]:
def validate_xml(xsd_file, xml_file) -> str:
    try:
        # Parse the XSD file
        with open(xsd_file, 'rb') as schema_file:
            schema_root = etree.XML(schema_file.read())
        schema = etree.XMLSchema(schema_root)

        # Parse the XML file
        with open(xml_file, 'rb') as xml_file:
            xml_doc = etree.parse(xml_file)

        # Validate the XML file against the XSD schema
        schema.assertValid(xml_doc)
        return "XML is valid."

    except etree.XMLSchemaError as e:
        return "XML is invalid: " + str(e)
    except Exception as e:
        return "An error occurred: " + str(e)

In [4]:
# messages.append(gpt_response)
# Example usage
xsd_file = 'app/resources/schema.xsd'
xml_file = 'app/gpt_outputs/gpt_example.xml'
ret = validate_xml(xsd_file, xml_file)
print(ret)
# messages.append(ret)

XML is valid.


## GeoJSON

In [ ]:
text: str = (
    """
{ 
"type": "FeatureCollection",
"features": [
    """
)

gps: str = (
    """
    {{
      "type": "Feature",
      "geometry": {{
        "type": "Point",
        "coordinates": [{}, {}]
      }},
      "properties": {{
        "marker-symbol": "{}-tree"
      }}
    }},"""
)

tree_type: str = "pistachio"
i = 0
dir = True
tree_num = 0

with open("resources/reza_waypoints.txt", "r") as fp:
    for line in fp.readlines():
        gps_coord = line.split(" ")
        # only want trees
        if gps_coord[2] == 0:
            continue
        if i % 18 == 0:
            dir = ~dir
            tree_num -= 18
        out = gps.format(gps_coord[0], gps_coord[1], tree_type)
        text += out
        i += 1

# remove last comma
text = text[:-1]
text += """  
  ]
}
"""

with open("resources/reza.geojson", "w") as fp:
    fp.write(text)